# Bilinear Interpolation
From Wikipedia: bilinear interpolation is a method for interpolating functions of two variables using linear interpolation. It is usually applied to 2D grids and this is what we will focus on.

## Principle
Bilinear interpolation is performed using linear interpolation first in one direction and then in the other direction.

# Linear interpolation
Linear interpolation is a method of curve fitting using linear polynomials to construct new data points within the range of two given known data points.
If the two points are given by the coordinates $(x_0, y_0)$ and $(x_1, y_1)$ then the linear interpolant is the straight line between these two points.

For an unknown point $x$ in the range, we can compute $y$ with 
$$ y = \frac{y_0(x_1 - x) + y_1(x-x_0)}{x_1 - x_0}$$


In [3]:
import inspect
from Interpolation.LinearInterpolation import LinearInterpolator
from ProjectiveGeometry.geometry import Point

print(inspect.getsource(LinearInterpolator))

class LinearInterpolator:
    def __init__(self, pointA: Point, pointB: Point):
        assert(pointA.x < pointB.x)
        self.pointA = pointA
        self.pointB = pointB
        
    def interpolate(self, x):
        num = self.pointA.y * (self.pointB.x - x) + self.pointB.y * (x - self.pointA.x)
        den = self.pointB.x - self.pointA.x
        return num / den



In [4]:
ptA = Point(0, 0)
ptB = Point(1, 1)
lnrI = LinearInterpolator(ptA, ptB)
lnrI.interpolate(0.5)

0.5